In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","")
v_file_date=dbutils.widgets.get("p_file_date")

In [0]:
# Presentation has delta 
race_results_df=spark.read.format('delta').load(f'{presentation_folder_path}/race_results')

In [0]:
# race_results_df=spark.read.parquet(f'{presentation_folder_path}/race_results')
# display(race_results_df)

In [0]:
from pyspark.sql.functions import *
drivers_standing_db=race_results_df.groupBy('race_year','driver_name','driver_nationality').agg(sum('points').alias('total_points'),count(when(col('position')==1,True)).alias('wins'))
display(drivers_standing_db)

In [0]:
display(drivers_standing_db.filter(drivers_standing_db.race_year==2020))

race_year,driver_name,driver_nationality,total_points,wins
2020,Kevin Magnussen,Danish,1.0,0
2020,Antonio Giovinazzi,Italian,4.0,0
2020,Pietro Fittipaldi,Brazilian,0.0,0
2020,Romain Grosjean,French,2.0,0
2020,Kimi Räikkönen,Finnish,4.0,0
2020,Lance Stroll,Canadian,75.0,0
2020,George Russell,British,3.0,0
2020,Valtteri Bottas,Finnish,223.0,2
2020,Daniel Ricciardo,Australian,119.0,0
2020,Charles Leclerc,Monegasque,98.0,0


In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.functions import col
from pyspark.sql import Window
final_drivers_standing=drivers_standing_db.withColumn('rank', rank().over(Window().partitionBy('race_year').orderBy(drivers_standing_db.total_points.desc(), drivers_standing_db.wins.desc()))).withColumn('file_date', lit(v_file_date))
display(final_drivers_standing.filter(col('race_year')==2020))

race_year,driver_name,driver_nationality,total_points,wins,rank,file_date
2020,Lewis Hamilton,British,347.0,11,1,2021-03-21
2020,Valtteri Bottas,Finnish,223.0,2,2,2021-03-21
2020,Max Verstappen,Dutch,214.0,2,3,2021-03-21
2020,Sergio Pérez,Mexican,125.0,1,4,2021-03-21
2020,Daniel Ricciardo,Australian,119.0,0,5,2021-03-21
2020,Alexander Albon,Thai,105.0,0,6,2021-03-21
2020,Carlos Sainz,Spanish,105.0,0,6,2021-03-21
2020,Charles Leclerc,Monegasque,98.0,0,8,2021-03-21
2020,Lando Norris,British,97.0,0,9,2021-03-21
2020,Pierre Gasly,French,75.0,1,10,2021-03-21


In [0]:
#final_drivers_standing.write.mode('overwrite').parquet(f'{presentation_folder_path}/drivers_standings')

In [0]:
#final_drivers_standing.write.mode('overwrite').format('parquet').saveAsTable('f1_presentation.drivers_standings')

In [0]:
#for delta 
merge_condition="tgt.race_year=src.race_year AND tgt.driver_name=src.driver_name"
merge_delta_data(final_drivers_standing,'f1_presentation','drivers_standings',presentation_folder_path,merge_condition,'race_year')

In [0]:
%sql
Select * from f1_presentation.drivers_standings

In [0]:
%sql
desc history f1_presentation.drivers_standings

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2025-01-28T07:53:56Z,5539008212065545,vishu.narang@outlook.com,MERGE,"Map(predicate -> [""((race_year#46217 = race_year#45970) AND (driver_name#46218 = driver_name#45974))""], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> false, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(2045452967433555),0104-100303-jdstnosa,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 72, numTargetBytesAdded -> 231379, numTargetBytesRemoved -> 231379, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 3142, executionTimeMs -> 11904, materializeSourceTimeMs -> 6499, numTargetRowsInserted -> 0, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 1124, numTargetRowsUpdated -> 3142, numOutputRows -> 3142, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 3142, numTargetFilesRemoved -> 72, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 4265)",null,Databricks-Runtime/15.4.x-scala2.12
1,2025-01-28T07:52:32Z,5539008212065545,vishu.narang@outlook.com,MERGE,"Map(predicate -> [""((race_year#43229 = race_year#42982) AND (driver_name#43230 = driver_name#42986))""], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> false, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(2045452967433555),0104-100303-jdstnosa,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 72, numTargetBytesAdded -> 231379, numTargetBytesRemoved -> 231379, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 3142, executionTimeMs -> 13076, materializeSourceTimeMs -> 6743, numTargetRowsInserted -> 0, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 1497, numTargetRowsUpdated -> 3142, numOutputRows -> 3142, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 3142, numTargetFilesRemoved -> 72, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 4793)",null,Databricks-Runtime/15.4.x-scala2.12
0,2025-01-28T07:51:25Z,5539008212065545,vishu.narang@outlook.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [""race_year""], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> false)",null,List(2045452967433555),0104-100303-jdstnosa,null,WriteSerializable,false,"Map(numFiles -> 72, numOutputRows -> 3142, numOutputBytes -> 231379)",null,Databricks-Runtime/15.4.x-scala2.12


In [0]:
dbutils.notebook.exit("Success")